# Time Series Machine Learning - Sheet 1
22/03/2024

Autors:
- Jesus David Mora Venegas
- Nicolay Agustin Cerda Cortez

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import scipy

## Exercise 1
1. Write a function `flatten_relative_and_full_ts_df(df)` that takes a dataframe, in which rows are subjects/instances, columns are features, and each cell is a sorted iterable (list, numpy array, Series). The output is a dataframe with a column for the subject (this can be just some integer, e.g, what previously used to be the index), one for the timestamp, and one other for each sensor.
This function assumes (and indicates in its name) that the time series objects are com- plete in the sense that even if a piece of information is missing, then this is indicated through a np.nan. Since the lists describing the time series have no timestamp yet, this timestamp is inferred implicitly from the observations’ index in the list.

In [41]:
df = pd.read_csv('db/ieeeppg.csv', delimiter=';')

for c in df:
    df[c] = df[c].apply(json.loads)

In [42]:
# for row_index,col in df[:1].iterrows():
#     for sensor, values in col.items():
#         for timestamp, value in enumerate(values):
#             print(row_index,timestamp)

In [43]:
def flatten_relative_and_full_ts_df(df):
    new_data = []
    # Convert into list
    for c in df:
        try:
            df[c] = df[c].apply(json.loads)
        except:
            print(f"The col {c} is allready a list.")
    # row_index = Index del row
    # col = ["Col name",[Values in col[row_index]]]
    for row_index, col in df.iterrows():
        print(type(col))
        # sendor = col_name
        # values = [values in col_name[row_index]]
        for sensor, values in col.items():
            # timestamp = index of value
            # value = sensor reading
            for timestamp, value in enumerate(values):
                if value == None:
                    value = np.nan
                new_data.append({
                    'subject': row_index,
                    'timestamp': timestamp,
                    'sensor': sensor,
                    'value': value
                })
                
    df_flatten = pd.DataFrame(new_data).pivot_table(index=['subject', 'timestamp'], columns='sensor', values='value').reset_index()
    return df_flatten
test = flatten_relative_and_full_ts_df(df)

The col PPG1 is allready a list.
The col PPG2 is allready a list.
The col x-accelerometer is allready a list.
The col y-accelerometer is allready a list.
The col z-accelerometer is allready a list.
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pa

In [44]:
test

sensor,subject,timestamp,PPG1,PPG2,x-accelerometer,y-accelerometer,z-accelerometer
0,0,0,-23.0,4.0,-0.0702,0.3432,0.9594
1,0,1,-24.0,6.0,-0.0702,0.3588,0.9438
2,0,2,-26.5,3.0,-0.0546,0.3666,0.9360
3,0,3,-27.0,3.5,-0.0468,0.3666,0.9360
4,0,4,-30.0,0.0,-0.0468,0.3744,0.9126
...,...,...,...,...,...,...,...
1767995,1767,995,-3.5,29.0,-0.0936,0.3900,0.6942
1767996,1767,996,4.0,34.0,-0.1014,0.3822,0.7020
1767997,1767,997,9.5,41.5,-0.1326,0.3978,0.7098
1767998,1767,998,18.5,49.0,-0.1404,0.4212,0.7332


2. Write a function `insert_times(df, timestamps, sub_col="x", time_col="timestamp")`
that generates, for each subject, all timestamps according to the time scheme defined in `timestamps`, which is a list or Series, and inserts respective rows into (a copy of) `df` with `np.nan` values for all datapoints.
This function assumes that `df` is in the flattened format. `sub_col` and `time_col` simply indicate the names of the columns that contain the subjects and the timestamps, respectively.

In [45]:
test['subject'].value_counts()

subject
0       1000
1174    1000
1185    1000
1184    1000
1183    1000
        ... 
586     1000
585     1000
584     1000
583     1000
1767    1000
Name: count, Length: 1768, dtype: int64

In [46]:
def insert_times(df, timestamps, sub_col="x", time_col="timestamp"):
    # Create a copy of the dataframe
    df_copy = df.copy()
    
    # If timestamps is a list, convert it to a Series
    if isinstance(timestamps, list):
        timestamps = pd.Series(timestamps)

    # Get unique subjects from the dataframe
    unique_subjects = df_copy[sub_col].unique()
    
    # Iterate over each subject
    for subject in unique_subjects:
        # Get the rows for the current subject
        subject_rows = df_copy[df_copy[sub_col] == subject]
        
        # Generate the missing timestamps for the subject
        missing_timestamps = timestamps[~timestamps.isin(subject_rows[time_col])]
        
        # Create new rows with np.nan values for the missing timestamps
        new_rows = pd.DataFrame({sub_col: subject, time_col: missing_timestamps})
        new_rows = new_rows.reindex(columns=df_copy.columns)
        
        # Append the new rows to the dataframe
        df_copy = pd.concat([df_copy, new_rows], ignore_index=True)
    
    return df_copy


Define timestamps

In [51]:
# ACTIVAR ESTE SI QUEREMOS UN TIMESTAMP DE FECHAS

#time1=pd.to_datetime('2020-01-01 00:00:00')
#time2 =time1 + pd.Timedelta(seconds=999)

# Generate the timestamps list using the defined time scheme
#timestamps = pd.date_range(start=time1, end=time2, freq='s').strftime('%Y-%m-%d %H:%M:%S').tolist()


In [47]:
timestamps=list(range(0,1200))

In [48]:
len(timestamps)

1200

In [49]:
#Ejecutar la función
df_copy = insert_times(test, timestamps, sub_col="subject", time_col="timestamp")

In [50]:
df_copy

sensor,subject,timestamp,PPG1,PPG2,x-accelerometer,y-accelerometer,z-accelerometer
0,0,0,-23.0,4.0,-0.0702,0.3432,0.9594
1,0,1,-24.0,6.0,-0.0702,0.3588,0.9438
2,0,2,-26.5,3.0,-0.0546,0.3666,0.9360
3,0,3,-27.0,3.5,-0.0468,0.3666,0.9360
4,0,4,-30.0,0.0,-0.0468,0.3744,0.9126
...,...,...,...,...,...,...,...
2121595,1767,1195,NaN,NaN,NaN,NaN,NaN
2121596,1767,1196,NaN,NaN,NaN,NaN,NaN
2121597,1767,1197,NaN,NaN,NaN,NaN,NaN
2121598,1767,1198,NaN,NaN,NaN,NaN,NaN
